In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from emonet import EmoNet,Modified_EmoNet
from torch.utils.data import DataLoader
import os
import re
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
import random
random.seed(42)
import time
from tqdm import tqdm

In [13]:
expw_df = pd.read_csv("D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig2\gpu2_env\expw_annotated.csv")
print("Class distribution:\n", expw_df['emotion'].value_counts())

Class distribution:
 emotion
amazed            3790
awe               3499
astound           1797
crying            1209
heartbroken       1081
angry             1070
fighting          1044
hostile            883
annoyed            797
fierce             761
mad                721
distressed         703
frightened         494
distaste           250
boring             240
disgust            238
anxious            224
expressionless     114
surprised           87
shocked             80
afraid              56
unhappy             39
ecstatic            32
excited             31
serious             17
horrified           15
fear                14
happy                3
joyous               1
Name: count, dtype: int64


In [14]:
print(expw_df['emotion'].unique())

['afraid' 'amazed' 'angry' 'annoyed' 'anxious' 'astound' 'awe' 'boring'
 'crying' 'disgust' 'distaste' 'distressed' 'ecstatic' 'excited'
 'expressionless' 'fear' 'fierce' 'fighting' 'frightened' 'happy'
 'heartbroken' 'horrified' 'hostile' 'joyous' 'mad' 'serious' 'shocked'
 'surprised' 'unhappy']


In [15]:
expw_to_emonet = {
    'afraid': 4,
    'amazed': 3,
    'angry': 6,
    'annoyed': 6,
    'anxious': 4,
    'astound': 3,
    'awe': 3,
    'boring': 0,
    'crying': 2,
    'disgust': 5,
    'distaste': 5,
    'distressed': 2,
    'ecstatic': 1,
    'excited': 1,
    'expressionless': 0,
    'fear': 4,
    'fierce': 6,
    'fighting': 6,
    'frightened': 4,
    'happy': 1,
    'heartbroken': 2,
    'horrified': 4,
    'hostile': 6,
    'joyous': 1,
    'mad': 6,
    'serious': 0,
    'shocked': 3,
    'surprised': 3,
    'unhappy': 2
}

expw_df['label'] = expw_df['emotion'].map(expw_to_emonet)
expw_df = expw_df.dropna(subset=['label']).reset_index(drop=True)
expw_df['label'] = expw_df['label'].astype(int)

In [16]:
expw_df.head()

,filename,emotion,race,gender,label
0,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,black,"{'Woman': np.float32(0.11790757), 'Man': np.fl...",4
1,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,black,"{'Woman': np.float32(0.24821858), 'Man': np.fl...",4
2,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(4.0783873), 'Man': np.flo...",4
3,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(99.97097), 'Man': np.floa...",4
4,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(99.9778), 'Man': np.float...",4


In [17]:
print("Class distribution:\n", expw_df['label'].value_counts())

Class distribution:
 label
3    9253
6    5276
2    3032
4     803
5     488
0     371
1      67
Name: count, dtype: int64


In [18]:
import re
import ast

def clean_and_parse_gender(raw):
    if isinstance(raw, str):
        cleaned = re.sub(r"np\.float32\((.*?)\)", r"\1", raw)
        return ast.literal_eval(cleaned)

# Apply it
expw_df['gender_dict'] = expw_df['gender'].apply(clean_and_parse_gender)


# Extract dominant gender
expw_df['dominant_gender'] = expw_df['gender_dict'].apply(lambda d: max(d, key=d.get))


In [19]:
expw_df['dominant_gender']

0          Man
1          Man
2          Man
3        Woman
4        Woman
         ...  
19285      Man
19286      Man
19287      Man
19288      Man
19289      Man
Name: dominant_gender, Length: 19290, dtype: object

In [20]:
expw_df.head()

,filename,emotion,race,gender,label,gender_dict,dominant_gender
0,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,black,"{'Woman': np.float32(0.11790757), 'Man': np.fl...",4,"{'Woman': 0.11790757, 'Man': 99.882095}",Man
1,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,black,"{'Woman': np.float32(0.24821858), 'Man': np.fl...",4,"{'Woman': 0.24821858, 'Man': 99.751785}",Man
2,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(4.0783873), 'Man': np.flo...",4,"{'Woman': 4.0783873, 'Man': 95.921616}",Man
3,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(99.97097), 'Man': np.floa...",4,"{'Woman': 99.97097, 'Man': 0.029033655}",Woman
4,D:\Integrated_gap_gradients\ig2_CNN\gpu_env_ig...,afraid,asian,"{'Woman': np.float32(99.9778), 'Man': np.float...",4,"{'Woman': 99.9778, 'Man': 0.022206737}",Woman


In [21]:
expw_df = expw_df.drop(columns=['gender', 'gender_dict'])
expw_df = expw_df.rename(columns={'dominant_gender': 'gender'})

In [22]:
expw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19290 entries, 0 to 19289
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  19290 non-null  object
 1   emotion   19290 non-null  object
 2   race      19290 non-null  object
 3   label     19290 non-null  int32 
 4   gender    19290 non-null  object
dtypes: int32(1), object(4)
memory usage: 678.3+ KB


In [23]:
expw_df['gender'] = expw_df['gender'].replace({'Man': 'male', 'Woman': 'female'})

In [24]:
keep_labels = [0, 1, 2, 4, 5, 6]
expw_df = expw_df[expw_df['label'].isin(keep_labels)].reset_index(drop=True)

In [25]:
print("Class distribution:\n", expw_df['label'].value_counts())

Class distribution:
 label
6    5276
2    3032
4     803
5     488
0     371
1      67
Name: count, dtype: int64


In [26]:
#creating pytorch dataset for expw annotated dataset
class ExpWAnnotatedDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'filename']
        label = self.df.loc[idx, 'label']
        race = self.df.loc[idx, 'race']
        gender = self.df.loc[idx, 'gender']

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label, race, gender, img_path


In [27]:
train_df, val_df = train_test_split(expw_df, test_size=0.3, stratify=expw_df['label'], random_state=42)

# Define image transforms
image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Create dataset instances
train_dataset = ExpWAnnotatedDataset(train_df, transform=image_transform)
val_dataset = ExpWAnnotatedDataset(val_df, transform=image_transform)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False,pin_memory=True)

In [28]:
#load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EmoNet(n_expression=8)
model.load_state_dict(torch.load("D:\Integrated_gap_gradients\ig2_CNN\emonet\pretrained\emonet_8.pth"))  # Adjust path if needed
model = model.to(device)

print(device)

cuda


In [15]:
"""for param in model.parameters():
    param.requires_grad = False

for param in model.emo_fc_2.parameters():
    param.requires_grad = True"""

'for param in model.parameters():\n    param.requires_grad = False\n\nfor param in model.emo_fc_2.parameters():\n    param.requires_grad = True'

In [29]:
for name, param in model.named_parameters():
    if "emo_fc_2" in name or "emo_net_2" in name or "m1" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

In [30]:
optimizer = torch.optim.SGD(
    filter(lambda p: p.requires_grad, model.parameters()), 
    lr=0.001, 
)
criterion = nn.CrossEntropyLoss()

In [18]:
"""import time

start_time = time.time()
num_epochs = 20
training_dict={}
training_dict["epochs"]=num_epochs
loss_list=[]
accuracy_list=[]

for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    correct, total = 0, 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad() 
        outputs = model(images)
        selected_logits = outputs["expression"]
        loss = criterion(selected_logits, labels)
        loss.backward() 
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(selected_logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        loss_list.append(loss)
        accuracy_list.append(round(100 * (correct / total), 2))

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")

end_time = time.time()
elapsed_time = end_time - start_time
training_dict["elapsed_time"]=elapsed_time
training_dict["loss"]=loss_list
training_dict["accuracy"]=accuracy_list

print(f"Execution Time: {elapsed_time:.4f} seconds")
torch.save(model.state_dict(), "expw_finetuned_emonet_20_epochs.pth")
print("Model saved!")"""

'import time\n\nstart_time = time.time()\nnum_epochs = 20\ntraining_dict={}\ntraining_dict["epochs"]=num_epochs\nloss_list=[]\naccuracy_list=[]\n\nfor epoch in range(num_epochs):\n    model.train() \n    running_loss = 0.0\n    correct, total = 0, 0\n    \n    for images, labels in train_loader:\n        images, labels = images.to(device), labels.to(device)\n\n        optimizer.zero_grad() \n        outputs = model(images)\n        selected_logits = outputs["expression"]\n        loss = criterion(selected_logits, labels)\n        loss.backward() \n        optimizer.step()\n        \n        running_loss += loss.item()\n        _, predicted = torch.max(selected_logits, 1)\n        total += labels.size(0)\n        correct += (predicted == labels).sum().item()\n        loss_list.append(loss)\n        accuracy_list.append(round(100 * (correct / total), 2))\n\n    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")\n\n

In [19]:
"""import time

start = time.time()
for i, (img, label, _, _, _) in enumerate(train_loader):
    if i >= 10:
        break
print(f"Time to load 10 batches: {time.time() - start:.2f} seconds")"""

'import time\n\nstart = time.time()\nfor i, (img, label, _, _, _) in enumerate(train_loader):\n    if i >= 10:\n        break\nprint(f"Time to load 10 batches: {time.time() - start:.2f} seconds")'

In [ ]:
start_time = time.time()
num_epochs = 50
training_dict = {
    "epochs": num_epochs,
    "loss": [],
    "accuracy": []
}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for images, labels, _, _, _ in loop:
        images, labels = images.to(device), labels.to(device).long()

        optimizer.zero_grad()
        outputs = model(images)
        selected_logits = outputs["expression"]
        loss = criterion(selected_logits, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(selected_logits, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        loop.set_postfix(loss=loss.item(), acc=100. * correct / total)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total
    training_dict["loss"].append(epoch_loss)
    training_dict["accuracy"].append(round(epoch_acc, 2))

    print(f"Epoch [{epoch+1}/{num_epochs}] ➤ Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")

end_time = time.time()
training_dict["elapsed_time"] = end_time - start_time
print(f"Total Training Time: {training_dict['elapsed_time'] / 60:.2f} minutes")

torch.save(model.state_dict(), "expw_finetuned_emonet_50_epochs_3unfreezed.pth")
print("Model saved!")

Epoch 1/50: 100%|██████████| 220/220 [02:56<00:00,  1.25it/s, acc=22.5, loss=7.22]


Epoch [1/50] ➤ Loss: 8.0786 | Accuracy: 22.55%


Epoch 2/50: 100%|██████████| 220/220 [02:17<00:00,  1.60it/s, acc=28.5, loss=5.3] 


Epoch [2/50] ➤ Loss: 5.3885 | Accuracy: 28.46%


Epoch 3/50: 100%|██████████| 220/220 [02:18<00:00,  1.59it/s, acc=33.4, loss=4.09]


...

Epoch 24/50: 100%|██████████| 220/220 [02:19<00:00,  1.58it/s, acc=54.9, loss=1.12] 


Epoch [24/50] ➤ Loss: 1.3130 | Accuracy: 54.89%


Epoch 25/50: 100%|██████████| 220/220 [02:19<00:00,  1.57it/s, acc=54.7, loss=1.25] 


Epoch [25/50] ➤ Loss: 1.2997 | Accuracy: 54.73%


Epoch 26/50:  51%|█████▏    | 113/220 [01:13<01:12,  1.47it/s, acc=55.4, loss=1.27]